# Robustness Evaluation of Pre-trained Language Models in the Russian Language


This is a notebook with experiments for "*Robustness Evaluation of Pre-trained Language Models in the Russian Language*"

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1KFoDzjdJZpZO55lKBHeH2yV2FWO_dS4E?usp=sharing)

[![View Source on GitHub](https://img.shields.io/badge/github-view%20source-black.svg)](https://github.com/yuliya1324/ru_attacker/blob/main/Experiments.ipynb)

In this notebook 7 attacks are performed. In order not to get lost, all of them are hidden in toggle lists, so you can view or try yourself only those that you are interested in.

But here are some tips:
* The design of backtranslation and paraphrasing is similar. Only accuracy and success rate are measured. Additionally, the dataframe of successful attacks is illustrated.
* Word order and arguments shuffling are also alike. Apart from accuracy and success rate, there are confusion matrices
* Yoda style transfer and proverb insertion illustrates only the process of attack itself
* Premise summarization is the fulliest part. Several hypotheses are tested in this section (**disclaimer**: none of which was actually confirmed)

Feel free to reproduce all the experiments. Or attack your own model using `BasicModel` or any of implemented classes. Or construct your own attacks with the help of `BasicAttack`.

# Installs

Install `ru_attacker`

In [ ]:
!pip install ru-attacker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Import classes for models and attacks from `ru_attacker`, also a utility function `get_data` to set the dataset, and some other packages

In [ ]:
from ru_attacker.ru_attacker.models import RobertaModel, T5Model
from ru_attacker.ru_attacker.attacks import (
    AddProverb,
    BackTranslation,
    ChangeArguments,
    Paraphrase,
    Summarization,
    WordOrder,
    YodaStyle,
    )
from ru_attacker.ru_attacker.models.set_dataset import get_data

import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Set models & get data

Set models from checkpoints

In [ ]:
roberta_checkpoints =  "ruRoberta-large"   # checkpoints to Roberta model
t5_checkpoints =  "rut5-large"  # checkpoints to T5 model

In [ ]:
ruRoberta = RobertaModel(roberta_checkpoints)
ruT5 = T5Model(t5_checkpoints)

Set the dataset, with the help of which models will be attacked

In [ ]:
data_dir = "TERRa/val.jsonl"  # path to data

In [ ]:
data = get_data(data_dir)

# Validate models

Firstly, validate models on this dataset to view the inital scores

In [ ]:
print(f'Accuracy for T5: {round(ruT5.validate(data_dir), 2)}')

100%|██████████| 39/39 [00:11<00:00,  3.53it/s]


0.83

In [ ]:
print(f'Accuracy for Roberta: {round(ruRoberta.validate(data_dir), 2)}')

100%|██████████| 20/20 [00:32<00:00,  1.63s/it]


0.84

# Attack models

### Backtranslation attack

First attack is backtranslation. In this attack and in paraphrasing only accuracy and success rate are measured, the rest of analysis is a hand validation of successful attacks, so here only attack process and examples of succeeded ones are illustrated. 

Now init backtranslation attack

In [ ]:
backtranslation_attack = BackTranslation()

Downloading:   0%|          | 0.00/908 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.80G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.54M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/272 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Attack ruT5

In [ ]:
results_backtranslation_t5 = backtranslation_attack.attack(ruT5, data)
backtranslation_df_t5 = pd.DataFrame(results_backtranslation_t5)


                  [Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:
                  entailment --> entailment
                  original premise: """Решение носит символический характер, так как взыскать компенсацию практически невозможно"", - отмечается в сообщении."
                  original hypothesis: Взыскать компенсацию не получится.
                  
                  transformed: Оплата компенсации не может быть получена.
                  
                  

                  [Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:
                  entailment --> not_entailment
                  original premise: "В суде Центрального района Сочи завершилось рассмотрение уголовного дела в отношении 81-летнего местного жителя, который застрелил обидчика своей дочери. ""Степан Вагин приговорен к трем годам лишения свободы условно с испытательным сроком два года"", - сообщил агентству ""Интерфакс-Юг"" в пятницу адвокат подсудимого Илья Иванов. По его словам, суд сделал сни

Here are successful attacks

In [ ]:
backtranslation_df_t5[backtranslation_df_t5["attack"] == "succeeded"].shape

(88, 7)

In [ ]:
backtranslation_df_t5[backtranslation_df_t5["attack"] == "succeeded"]

,Unnamed: 0,original label,attacked label,original premise,original hypothesis,transformed,attack
1,1,1,0.0,"""В суде Центрального района Сочи завершилось р...",81-летний пенсионер застрелил обидчика дочери.,81-летний пенсионер убил жертву своей дочери.,succeeded
5,5,1,0.0,"Кроме того, власти Грузии подписали на доброво...",Власти Грузии полностью выполняют подписанные ...,Власти Грузии полностью придерживаются подписа...,succeeded
12,12,0,1.0,"Концерт стал знаменателен и тем, что во время ...",Во время концертов певица всегда презентует св...,На концертах певица показывает свои новые альб...,succeeded
13,13,1,0.0,Молюсь за обоих. Просто хочу обнять своего сын...,Кто-то оскорбил милицию.,Кто-то злоупотреблял полицией.,succeeded
14,14,0,1.0,"Документы снова вернутся на рассмотрение, поск...",Верховный суд завернул апелляцию.,Верховный суд принял решение об апелляции.,succeeded
...,...,...,...,...,...,...,...
242,242,0,1.0,"Кроме того, в 2012 году из бюджета было выделе...",В 2012 году региональные авиакомпании принесли...,В 2012 году региональные авиакомпании принесли...,succeeded
243,243,1,0.0,"""Адвокаты обвиняемых попросили оправдать их по...",Защитник хочет снять с обвиняемой обвинение в ...,Защитник хочет отнять обвиняемого в убийстве.,succeeded
245,245,1,0.0,Как пояснили в пресс-службе городской админист...,«Сложение потенциалов» - это создание замкнуты...,«Один потенциал» — это создание закрытых цикло...,succeeded
246,246,0,1.0,Ежедневно в плотины поступает всё больше и бол...,Каждый день в плотины поступает горячая вода.,Ежедневно в плато приходит горячая вода.,succeeded


Attack Roberta

In [ ]:
results_backtranslation_roberta = backtranslation_attack.attack(ruRoberta, data)
backtranslation_df_roberta = pd.DataFrame(results_backtranslation_roberta)


                  [Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:
                  entailment --> not_entailment
                  original premise: """Решение носит символический характер, так как взыскать компенсацию практически невозможно"", - отмечается в сообщении."
                  original hypothesis: Взыскать компенсацию не получится.
                  
                  transformed: Компенсации не будет требоваться.
                  
                  

                  [Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:
                  entailment --> not_entailment
                  original premise: Об этом вечером во вторник, 17 января, сообщила пресс-служба Спасательного департамента, отметив, что немецкую противотанковую мину Tellermine 42 обнаружили в на улице Кеэвисе в ходе земляных работ. Спасатели эвакуировали жителей окрестных домов, офисов и складских помещений. Уничтожать мину на месте не стали, поскольку это угрожало повреждению трассы трубопровода.

And the same thing for Roberta

In [ ]:
backtranslation_df_roberta[backtranslation_df_roberta["attack"] == "succeeded"].shape

(98, 7)

In [ ]:
backtranslation_df_roberta[backtranslation_df_roberta["attack"] == "succeeded"]

,Unnamed: 0,original label,attacked label,original premise,original hypothesis,transformed,attack
0,0,1,0.0,"""""""Решение носит символический характер, так к...",Взыскать компенсацию не получится.,Компенсации не будет требоваться.,succeeded
1,1,1,0.0,"Об этом вечером во вторник, 17 января, сообщил...",На улице Кеэвисе жителей эвакуировали из-за мины.,На улице Кевиса жители эвакуировали из-за шахт.,succeeded
2,2,1,0.0,"""В суде Центрального района Сочи завершилось р...",81-летний пенсионер застрелил обидчика дочери.,81-летний мужчина застрелил дочь.,succeeded
10,10,1,0.0,"""Военнослужащие ВМС Израиля в субботу высадили...",Военные захватили контроль над судном.,Военные завоевали контроль над судом.,succeeded
13,13,0,1.0,"Концерт стал знаменателен и тем, что во время ...",Во время концертов певица всегда презентует св...,Во время концертов певица показывает свои новы...,succeeded
...,...,...,...,...,...,...,...
245,245,0,1.0,Ежедневно в плотины поступает всё больше и бол...,Каждый день в плотины поступает горячая вода.,Каждый день есть горячая вода.,succeeded
247,247,1,0.0,"""Около 200 человек эвакуированы из корпуса фак...",Трансформаторная подстанция загорелась.,Трансформаторная подстанция исчезла.,succeeded
248,248,1,0.0,"Между тем в кредитной организации заверили, чт...",В кредитной организации приуменьшили масштаб п...,В кредитной организации уменьшился масштаб про...,succeeded
253,253,1,0.0,"Грабитель поджидал своих жертв и, угрожая им п...",У грабителя был пистолет.,У кражья была пистолет.,succeeded


### Paraphrasing attack

Init a paraphrasing attack

In [ ]:
paraphrase_attack = Paraphrase()

Downloading:   0%|          | 0.00/724 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/932M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/808k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/315 [00:00<?, ?B/s]

Attack T5

In [ ]:
results_paraphrase_t5 = paraphrase_attack.attack(ruT5, data)
paraphrase_df_t5 = pd.DataFrame(results_paraphrase_t5)


                  [Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:
                  entailment --> entailment
                  original premise: """Решение носит символический характер, так как взыскать компенсацию практически невозможно"", - отмечается в сообщении."
                  original hypothesis: Взыскать компенсацию не получится.
                  
                  transformed: Не получится взыскать компенсации.
                  
                  

                  [Succeeded / Failed / Skipped / Total] 0 / 2 / 0 / 2:
                  entailment --> entailment
                  original premise: "В суде Центрального района Сочи завершилось рассмотрение уголовного дела в отношении 81-летнего местного жителя, который застрелил обидчика своей дочери. ""Степан Вагин приговорен к трем годам лишения свободы условно с испытательным сроком два года"", - сообщил агентству ""Интерфакс-Юг"" в пятницу адвокат подсудимого Илья Иванов. По его словам, суд сделал снисхождение пе

Examples of successful attacks:

In [ ]:
paraphrase_df_t5[paraphrase_df_t5["attack"] == "succeeded"].shape

(23, 7)

In [ ]:
paraphrase_df_t5[paraphrase_df_t5["attack"] == "succeeded"]

,Unnamed: 0,original label,attacked label,original premise,original hypothesis,transformed,attack
14,14,0,1.0,"Документы снова вернутся на рассмотрение, поск...",Верховный суд завернул апелляцию.,В Верховном Суде было занесено апелляционное з...,succeeded
18,18,0,1.0,Мы попытались это сделать в МГУ. Сначала проло...,"Сперва мы проложили по дороге, затем оставили.","Мы сначала проложили дорогу, а потом оставили.",succeeded
21,21,0,1.0,"""В кассах с утра продавали билеты без предъявл...",Кассиры сознательно нарушали закон о сохранени...,Кассиры соответственно нарушили закон о защите...,succeeded
31,31,0,1.0,"Приятная новость для тех, кто задумывался об и...",Есть плохая новость.,Хорошая новость есть.,succeeded
35,35,0,1.0,Компенсации полагаются каждому из 4234 пассажи...,Не каждому из пассажиров положена компенсация.,Не каждый пассажир получает компенсацию.,succeeded
50,50,0,1.0,К спасательной операции подключились силы Наци...,В Европе наводнение.,В Европе есть наводнения.,succeeded
54,54,1,0.0,"Рождество Христово - время радости, и этой рад...",Рождество - радостное время.,Рождество – это веселое время.,succeeded
74,74,1,0.0,"""ЛГБТ-активисту Николаю Алексееву, который был...","Адвокат Генри Резник считает, что в Конституци...","Адвокат Генрих Резник полагает, что активист Л...",succeeded
88,88,1,0.0,В городе Ветлуга Нижегородской области на пожа...,Все спали и не видели пожара.,"Все спали, и пожара не видели.",succeeded
90,90,1,0.0,Ведь тут 50 на 50. Ты приехал перед олимпийски...,Отсутствие снега может заставить спортсмена ки...,Отсутствие снежного покрова может заставлять с...,succeeded


Attack Roberta

In [ ]:
results_paraphrase_roberta = paraphrase_attack.attack(ruRoberta, data)
paraphrase_df_roberta = pd.DataFrame(results_paraphrase_roberta)


                  [Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:
                  entailment --> entailment
                  original premise: """Решение носит символический характер, так как взыскать компенсацию практически невозможно"", - отмечается в сообщении."
                  original hypothesis: Взыскать компенсацию не получится.
                  
                  transformed: Не получится взыскать компенсации.
                  
                  

                  [Succeeded / Failed / Skipped / Total] 0 / 1 / 1 / 2:
                  entailment --> Skipped

                  

                  [Succeeded / Failed / Skipped / Total] 1 / 1 / 1 / 3:
                  entailment --> Skipped

                  

                  [Succeeded / Failed / Skipped / Total] 1 / 2 / 1 / 4:
                  entailment --> entailment
                  original premise: "Кто-то нарисовал на снегу контур человеческого тела, как делают на месте убийств в сериалах о полиции, и 

Successful attacks

In [ ]:
paraphrase_df_roberta[paraphrase_df_roberta["attack"] == "succeeded"].shape

(29, 7)

In [ ]:
paraphrase_df_roberta[paraphrase_df_roberta["attack"] == "succeeded"]

,Unnamed: 0,original label,attacked label,original premise,original hypothesis,transformed,attack
2,2,1,0.0,"""В суде Центрального района Сочи завершилось р...",81-летний пенсионер застрелил обидчика дочери.,81-летняя пенсионерка застрелила дочь.,succeeded
14,14,0,1.0,"Документы снова вернутся на рассмотрение, поск...",Верховный суд завернул апелляцию.,В Верховном Суде было занесено апелляционное з...,succeeded
24,24,1,0.0,"Народ уже задумывается, куда отправится повесе...",Народ готовится повеселиться.,Народ готов веселиться.,succeeded
34,34,0,1.0,"Приятная новость для тех, кто задумывался об и...",Есть плохая новость.,Хорошая новость есть.,succeeded
52,52,0,1.0,К спасательной операции подключились силы Наци...,В Европе наводнение.,В Европе есть наводнения.,succeeded
61,61,0,1.0,Учредителем центра стала Государственная канце...,Государственная канцелярия выступила перед учр...,Перед учредителем выступила государственная ка...,succeeded
72,72,1,0.0,В случившееся трудно поверить. Произошла огром...,Люди погибли и их не вернешь.,"Люди умерли, и они не вернутся.",succeeded
78,78,1,0.0,"""ЛГБТ-активисту Николаю Алексееву, который был...","Адвокат Генри Резник считает, что в Конституци...","Адвокат Генрих Резник полагает, что активист Л...",succeeded
95,95,1,0.0,Как рассказали «Комсомольской правде» в пресс-...,Новую автобусную остановку устанавливают.,Установлены новые автобусные остановки.,succeeded
99,99,1,0.0,"""""""Потому что вокруг этой фигуры в Интернете п...",Медведев отметил полемику в Интернете вокруг э...,Медведев назвал полемикой в интернете эту фигуру.,succeeded


### Premise summarization attack

In [ ]:
summarization_attack = Summarization()

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.23G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/287 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406 [00:00<?, ?B/s]

In [ ]:
results_summarization_t5 = summarization_attack.attack(ruT5, data)
summarization_df_t5 = pd.DataFrame(results_summarization_t5)


                  [Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:
                  entailment --> Skipped

                  


/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:998: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,



                  [Succeeded / Failed / Skipped / Total] 0 / 1 / 1 / 2:
                  entailment --> entailment
                  original premise: "В суде Центрального района Сочи завершилось рассмотрение уголовного дела в отношении 81-летнего местного жителя, который застрелил обидчика своей дочери. ""Степан Вагин приговорен к трем годам лишения свободы условно с испытательным сроком два года"", - сообщил агентству ""Интерфакс-Юг"" в пятницу адвокат подсудимого Илья Иванов. По его словам, суд сделал снисхождение пенсионеру, поскольку он является тружеником тыла и инвалидом второй группы, и назначил наказание ""ниже низшего предела""."
                  original hypothesis: 81-летний пенсионер застрелил обидчика дочери.
                  
                  transformed: В Сочи завершилось рассмотрение уголовного дела в отношении 81-летнего местного жителя, который застрелил обидчика своей дочери. Он приговорен к трем годам условно с испытательным сроком два года.
                 

Your max_length is set to 50, but you input_length is only 39. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)



                  [Succeeded / Failed / Skipped / Total] 18 / 28 / 40 / 86:
                  entailment --> Skipped

                  

                  [Succeeded / Failed / Skipped / Total] 19 / 28 / 40 / 87:
                  not_entailment --> entailment
                  original premise: Это предполагает законопроект о фонде субсидирования предприятий, которые создают новые рабочие места, рассмотренный комиссией по сельскому хозяйству и пищевой промышленности и предложенный парламенту.
                  original hypothesis: Законопроект завернула комиссия по сельскому хозяйству.
                  
                  transformed: В Госдуму внесен законопроект о фонде субсидирования предприятий, которые создают новые рабочие места. Это предполагает законопроект о фонде субсидирования предприятий, которые создают новые рабочие места, рассмотренный комиссией по сельскому хозяйству и пище
                  
                  

                  [Succeeded / Failed / Skipped / Total

...


                  [Succeeded / Failed / Skipped / Total] 57 / 57 / 96 / 210:
                  entailment --> Skipped

                  

                  [Succeeded / Failed / Skipped / Total] 58 / 57 / 96 / 211:
                  not_entailment --> entailment
                  original premise: По итогам работы круглого стола, на котором сегодня был рассмотрен проект Стратегии социально-экономического развития Иркутской области, будут выработаны рекомендации для достижения целей, сформулированных в документе.
                  original hypothesis: Будут отменены рекомендации к целям Стратегии социально-экономического развития Иркутской области.
                  
                  transformed: По итогам круглого стола, на котором сегодня был рассмотрен проект Стратегии социально-экономического развития Иркутской области, будут выработаны рекомендации для достижения целей, сформулированных в документе.
                  
                  

                  [Succeeded / Failed / S

Your max_length is set to 50, but you input_length is only 46. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)



                  [Succeeded / Failed / Skipped / Total] 59 / 59 / 99 / 217:
                  entailment --> entailment
                  original premise: Я давно хотела слепить снежный мотоцикл, но вот мы только собрались, примерились к этой куче, а я глянула и поняла, что вижу там только Будду. Доброго такого, пузатого, довольного и умиротворенного, - рассказывает Дарья. - Пока делали, несколько раз приходилось объяснять соседям, что это не развратная женщина, потому что начали с ног.
                  original hypothesis: Дарья хотела слепить из снега мотоцикл, а слепила Будду.
                  
                  transformed: На выставке слепить снежный мотоцикл Дарья рассказала о том, как она видит Будду.
                  
                  

                  [Succeeded / Failed / Skipped / Total] 59 / 60 / 99 / 218:
                  entailment --> entailment
                  original premise: Средства на пополнение парка малой авиации Приморского края обещали выделить из р

Your max_length is set to 50, but you input_length is only 46. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)



                  [Succeeded / Failed / Skipped / Total] 59 / 63 / 103 / 225:
                  not_entailment --> Skipped

                  

                  [Succeeded / Failed / Skipped / Total] 60 / 63 / 103 / 226:
                  not_entailment --> entailment
                  original premise: Средства на пополнение парка малой авиации Приморского края обещали выделить из регионального бюджета. Машины будут приобретены в лизинг. Как сообщили в администрации края, сделка находится на завершающей стадии.
                  original hypothesis: Машины приобретать не станут.
                  
                  transformed: Финансирование пополнения парка малой авиации Приморского края обещали выделить из регионального бюджета. Машины будут приобретены в лизинг. Как сообщили в администрации края, сделка находится на завершающей стадии.
                  
                  

                  [Succeeded / Failed / Skipped / Total] 60 / 63 / 104 / 227:
                  entailment

In [ ]:
results_summarization_roberta = summarization_attack.attack(ruRoberta, data)
summarization_df_roberta = pd.DataFrame(results_summarization_roberta)


                  [Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:
                  entailment --> Skipped

                  

                  [Succeeded / Failed / Skipped / Total] 0 / 1 / 1 / 2:
                  entailment --> entailment
                  original premise: Об этом вечером во вторник, 17 января, сообщила пресс-служба Спасательного департамента, отметив, что немецкую противотанковую мину Tellermine 42 обнаружили в на улице Кеэвисе в ходе земляных работ. Спасатели эвакуировали жителей окрестных домов, офисов и складских помещений. Уничтожать мину на месте не стали, поскольку это угрожало повреждению трассы трубопровода.
                  original hypothesis: На улице Кеэвисе жителей эвакуировали из-за мины.
                  
                  transformed: По предварительным данным, немецкую противотанковую мину Tellermine 42 обнаружили в на улице Кеэвисе в ходе земляных работ. Уничтожать мину на месте не стали, поскольку это угрож
                  
        

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:998: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,



                  [Succeeded / Failed / Skipped / Total] 4 / 7 / 6 / 17:
                  entailment --> entailment
                  original premise: "Сулейманов подчеркнул, что по тонкому льду нельзя ездить на таком примитивном средстве, как мотобуксировщик. Если снегоход еще может проскочить опасное место, то ""мотособака"" тянет как якорь в любую полынью. - Но рыбаки и дачники активно используют эти машины, потому что стоят они недорого, просты в управлении - к примитивному мотору с ручкой цепляются любые сани."
                  original hypothesis: Сулейманов объяснил, чем мотобуксировщик опаснее снегохода.
                  
                  transformed: Российские рыбаки и дачники активно используют эти машины, потому что стоят они недорого, просты в управлении - к примитивному мотору с ручкой цепляются любые сани.
                  
                  

                  [Succeeded / Failed / Skipped / Total] 5 / 7 / 6 / 18:
                  not_entailment --> entailment
 

Your max_length is set to 50, but you input_length is only 39. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)



                  [Succeeded / Failed / Skipped / Total] 17 / 32 / 43 / 92:
                  not_entailment --> entailment
                  original premise: Это предполагает законопроект о фонде субсидирования предприятий, которые создают новые рабочие места, рассмотренный комиссией по сельскому хозяйству и пищевой промышленности и предложенный парламенту.
                  original hypothesis: Законопроект завернула комиссия по сельскому хозяйству.
                  
                  transformed: В Госдуму внесен законопроект о фонде субсидирования предприятий, которые создают новые рабочие места. Это предполагает законопроект о фонде субсидирования предприятий, которые создают новые рабочие места, рассмотренный комиссией по сельскому хозяйству и пище
                  
                  

                  [Succeeded / Failed / Skipped / Total] 17 / 32 / 44 / 93:
                  not_entailment --> Skipped

                  

                  [Succeeded / Failed / Skipped / T

...


                  [Succeeded / Failed / Skipped / Total] 47 / 67 / 94 / 208:
                  entailment --> Skipped

                  

                  [Succeeded / Failed / Skipped / Total] 48 / 67 / 94 / 209:
                  not_entailment --> entailment
                  original premise: По итогам работы круглого стола, на котором сегодня был рассмотрен проект Стратегии социально-экономического развития Иркутской области, будут выработаны рекомендации для достижения целей, сформулированных в документе.
                  original hypothesis: Будут отменены рекомендации к целям Стратегии социально-экономического развития Иркутской области.
                  
                  transformed: По итогам круглого стола, на котором сегодня был рассмотрен проект Стратегии социально-экономического развития Иркутской области, будут выработаны рекомендации для достижения целей, сформулированных в документе.
                  
                  

                  [Succeeded / Failed / S

Your max_length is set to 50, but you input_length is only 46. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)



                  [Succeeded / Failed / Skipped / Total] 51 / 68 / 98 / 217:
                  entailment --> entailment
                  original premise: Я давно хотела слепить снежный мотоцикл, но вот мы только собрались, примерились к этой куче, а я глянула и поняла, что вижу там только Будду. Доброго такого, пузатого, довольного и умиротворенного, - рассказывает Дарья. - Пока делали, несколько раз приходилось объяснять соседям, что это не развратная женщина, потому что начали с ног.
                  original hypothesis: Дарья хотела слепить из снега мотоцикл, а слепила Будду.
                  
                  transformed: На выставке слепить снежный мотоцикл Дарья рассказала о том, как она видит Будду.
                  
                  

                  [Succeeded / Failed / Skipped / Total] 51 / 69 / 98 / 218:
                  entailment --> entailment
                  original premise: Средства на пополнение парка малой авиации Приморского края обещали выделить из р

Your max_length is set to 50, but you input_length is only 46. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)



                  [Succeeded / Failed / Skipped / Total] 51 / 72 / 102 / 225:
                  not_entailment --> Skipped

                  

                  [Succeeded / Failed / Skipped / Total] 52 / 72 / 102 / 226:
                  not_entailment --> entailment
                  original premise: Средства на пополнение парка малой авиации Приморского края обещали выделить из регионального бюджета. Машины будут приобретены в лизинг. Как сообщили в администрации края, сделка находится на завершающей стадии.
                  original hypothesis: Машины приобретать не станут.
                  
                  transformed: Финансирование пополнения парка малой авиации Приморского края обещали выделить из регионального бюджета. Машины будут приобретены в лизинг. Как сообщили в администрации края, сделка находится на завершающей стадии.
                  
                  

                  [Succeeded / Failed / Skipped / Total] 52 / 72 / 103 / 227:
                  entailment

#### Hypothesis №1

The summarised texts might contain all the relevant information but be shorter, so it makes the task for the model easier as it takes into consideration less information.

To test this hypothesis we constructed new class which outputs those samples in which true label is *entailment*, and, firstly, model did not detect it correctly. But after summarization it changed its prediction as it could reason from a shorter text better.

In [ ]:
class Summarization_custom(Summarization):
    def attack(self, model, dataset):
        for i, row in dataset.iterrows():
            premise = row["premise"]
            hypothesis = row["hypothesis"]
            label = row["label"]
            if len(premise) < 200:
                continue
            if label == 1:
                prediction = model.predict(model.prepare_data(premise, hypothesis))
                if prediction == 0:
                    transformed = self.summarizer(premise, max_length=50)[0]["summary_text"]
                    prediction = model.predict(model.prepare_data(premise, transformed))
                    if prediction == 1:
                        print(f"""
Original premise: {premise}
Hypothesis: {hypothesis}

Summarised premise: {transformed}
                        """)

In [ ]:
summarisation = Summarization_custom()

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.23G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/287 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406 [00:00<?, ?B/s]

In [ ]:
summarisation.attack(ruT5, data)


Original premise: Об этом вечером во вторник, 17 января, сообщила пресс-служба Спасательного департамента, отметив, что немецкую противотанковую мину Tellermine 42 обнаружили в на улице Кеэвисе в ходе земляных работ. Спасатели эвакуировали жителей окрестных домов, офисов и складских помещений. Уничтожать мину на месте не стали, поскольку это угрожало повреждению трассы трубопровода.
Hypothesis: На улице Кеэвисе жителей эвакуировали из-за мины.

Summarised premise: По предварительным данным, немецкую противотанковую мину Tellermine 42 обнаружили в на улице Кеэвисе в ходе земляных работ. Уничтожать мину на месте не стали, поскольку это угрож
                        

Original premise: Разработчик видеоигр индийского происхождения Шекхар, живущий в Лондоне, пытается привить своему десятилетнему сыну Пратику доброту и мудрость, которыми сам обладает в достаточном количестве. Сын же помешан на всем, что связано с технологиями, и в частности на видеоиграх, и отвлекаться на подобную чепуху н

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:998: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,



Original premise: Трижды он был привлечён судебным приставом к административной ответственности по ст. 17.15 КоАП РФ за неисполнение содержащихся в исполнительном документе требований неимущественного характера. Так как срок для добровольного исполнения истёк, пристрой снесли принудительно.
Hypothesis: Пристроя больше нет.

Summarised premise: Российский судебный пристав, который дважды привлекал истёка к административной ответственности за неисполнение содержащихся в исполнительном документе требований неимущественного характера, снёс пристрой принудительно
                        

Original premise: "Однако отсутствие на форуме нового вице-премьера, который предпочел Рио-де-Жанейро Санкт-Петербургу, и бурная активность вице-премьера бывшего - нынешнего президента ""Роснефти"" Игоря Сечина, центральной фигуры непрекращающихся встреч с иностранными инвесторами, прояснили, кто тут главный. "
Hypothesis: Сечин был центральной фигурой встреч с иностранными инвесторами.

Summarised premis

In [ ]:
summarisation.attack(ruRoberta, data)

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:998: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,



Original premise: Разработчик видеоигр индийского происхождения Шекхар, живущий в Лондоне, пытается привить своему десятилетнему сыну Пратику доброту и мудрость, которыми сам обладает в достаточном количестве. Сын же помешан на всем, что связано с технологиями, и в частности на видеоиграх, и отвлекаться на подобную чепуху не желает. Единственное чего он хочет, так это получить игру с непобедимым злодеем, потому что достиг уже такого уровня профессионализма в играх, что все остальное ему кажется скучным и неинтересным.
Hypothesis: Сын разработчика видеоигр не ценит доброту и мудрость.

Summarised premise: Разработчик видеоигр индийского происхождения Шекхар, живущий в Лондоне, пытается привить своему десятилетнему сыну доброту и мудрость, которыми сам обладает в достаточном количестве.
                        

Original premise: Преступление удалось раскрыть в кратчайшие сроки благодаря тому, что пострадавшая сразу сообщила о случившемся в полицию. На помощь женщине оперативный дежурны

The most part of samples are not valid: the summarized texts do not contain all the relevant information required to make the right prediction.

The hypothesis is **rejected**

#### Hypothesis № 2

The summarised text might miss some crucial information, so in cases where the entailment was present between original inputs it cannot be reasoned anymore.

**Test 1**. A good model would lose the connection of entailment where it was initially present

In [ ]:
summarization_df_t5[(summarization_df_t5["original label"] == 1) & (summarization_df_t5["attacked label"] == 0)]

,Unnamed: 0,original label,attacked label,original premise,original hypothesis,transformed,attack
5,5,1,0.0,"Кроме того, власти Грузии подписали на доброво...",Власти Грузии полностью выполняют подписанные ...,Накануне встречи с представителями грузинских ...,succeeded
26,26,1,0.0,"Мне очень жаль, если мои слова восприняли как ...",Я желаю русскому хоккею светлого будущего.,Форвард сборной России по хоккею Евгений Малки...,succeeded
96,96,1,0.0,В роли сказочного персонажа также был полицейс...,Полицейский нарядился Дедом Морозом.,"Сказочный полицейский, облаченный в красную шу...",succeeded
165,165,1,0.0,"Количество прихожан в той небольшой часовне, к...",Прихожан в часовне на Сенной все больше.,"Епархия просит прощения у прихожан за то, что ...",succeeded
167,167,1,0.0,"Игроки хорошие, индивидуумы хорошие, а не слож...","Президень искала лучшее, что есть на рынке.","В среду состоялась пресс-конференция, на котор...",succeeded
188,188,1,0.0,На коробках с пекинской капустой и яблоками на...,Пекинская капуста приехала из Польши.,Российская продукция пересекла границу Республ...,succeeded


In [ ]:
summarization_df_roberta[(summarization_df_roberta["original label"] == 1) & (summarization_df_roberta["attacked label"] == 0)]

,Unnamed: 0,original label,attacked label,original premise,original hypothesis,transformed,attack
6,6,1,0.0,"Кроме того, власти Грузии подписали на доброво...",Власти Грузии полностью выполняют подписанные ...,Накануне встречи с представителями грузинских ...,succeeded
23,23,1,0.0,"2 августа пьяные приятели, шагая по грунтовке,...",Неподалеку застрял автомобиль их подруги.,В Москве разбили стекло «Нивы» пьяные приятели...,succeeded
89,89,1,0.0,"Как утверждает сама Якуб, она познакомилась с ...",Потерпевшая дала показания.,В деле о изнасиловании горничной нью-йоркского...,succeeded
100,100,1,0.0,В роли сказочного персонажа также был полицейс...,Полицейский нарядился Дедом Морозом.,"Сказочный полицейский, облаченный в красную шу...",succeeded
164,164,1,0.0,"Количество прихожан в той небольшой часовне, к...",Прихожан в часовне на Сенной все больше.,"Епархия просит прощения у прихожан за то, что ...",succeeded
166,166,1,0.0,"Игроки хорошие, индивидуумы хорошие, а не слож...","Президень искала лучшее, что есть на рынке.","В среду состоялась пресс-конференция, на котор...",succeeded
200,200,1,0.0,Мужчины распивали алкоголь и вдруг поругались....,Мужчины поругались в процессе пьянки.,С ножом на кухне поругался один из собутыльник...,succeeded
209,209,1,0.0,"""""""Все три птицы чувствуют себя хорошо, ведут ...",В Ленинградском зоопарке три птицы этого вида.,Дневник Ленинградского зоопарка: три птицы чув...,succeeded
215,215,1,0.0,"""Однако отсутствие на форуме нового вице-премь...",Сечин был центральной фигурой встреч с иностра...,В Санкт-Петербурге завершился ежегодный междун...,succeeded
239,239,1,0.0,Руки подсудимого были забинтованы и находились...,Подсудимый был украинцем.,"На Украине судят украинца, который забинтовал ...",succeeded


Not much instances, not all of which are valid.

**Test 2**. There is missed information and the model is attacked, when the model insists on the entailment even without crucial information.

In [ ]:
summarization_df_t5[(summarization_df_t5["original label"] == 1) & (summarization_df_t5["attacked label"] == 1)]

,Unnamed: 0,original label,attacked label,original premise,original hypothesis,transformed,attack
1,1,1,1.0,"""В суде Центрального района Сочи завершилось р...",81-летний пенсионер застрелил обидчика дочери.,В Сочи завершилось рассмотрение уголовного дел...,failed
3,3,1,1.0,"Также установлено, что работники учреждения не...",В детском саду не все вовремя проходят медосмотр.,"В детском саду «Олимпийский» выявлено, что раб...",failed
6,6,1,1.0,"Сотрудники МЧС извлекли его из воды, после чег...",Спасатели достали мужчину из воды.,"Сотрудники МЧС извлекли его из воды, после чег...",failed
7,7,1,1.0,"Мы уже говорили, что подрядчики не должны этог...",Результат для нас в приоритете.,"Строители не будут оплачивать работы, которые ...",failed
9,9,1,1.0,"""Военнослужащие ВМС Израиля в субботу высадили...",Военные захватили контроль над судном.,"ВМС Израиля высадились на судно «Эстель», пере...",failed
...,...,...,...,...,...,...,...
241,241,1,1.0,Руки подсудимого были забинтованы и находились...,Подсудимый был украинцем.,"На Украине судят украинца, который забинтовал ...",failed
243,243,1,1.0,"""Адвокаты обвиняемых попросили оправдать их по...",Защитник хочет снять с обвиняемой обвинение в ...,Заместители обвиняемых в убийстве Сергея Шкапц...,failed
245,245,1,1.0,Как пояснили в пресс-службе городской админист...,«Сложение потенциалов» - это создание замкнуты...,"По мнению экспертов, в качестве конкурентного ...",failed
248,248,1,1.0,"""Около 200 человек эвакуированы из корпуса фак...",Трансформаторная подстанция загорелась.,В связи с пожаром в подвальном помещении факул...,failed


In [ ]:
summarization_df_roberta[(summarization_df_roberta["original label"] == 1) & (summarization_df_roberta["attacked label"] == 1)]

,Unnamed: 0,original label,attacked label,original premise,original hypothesis,transformed,attack
1,1,1,1.0,"Об этом вечером во вторник, 17 января, сообщил...",На улице Кеэвисе жителей эвакуировали из-за мины.,"По предварительным данным, немецкую противотан...",failed
2,2,1,1.0,"""В суде Центрального района Сочи завершилось р...",81-летний пенсионер застрелил обидчика дочери.,В Сочи завершилось рассмотрение уголовного дел...,failed
4,4,1,1.0,"Также установлено, что работники учреждения не...",В детском саду не все вовремя проходят медосмотр.,"В детском саду «Олимпийский» выявлено, что раб...",failed
7,7,1,1.0,"Сотрудники МЧС извлекли его из воды, после чег...",Спасатели достали мужчину из воды.,"Сотрудники МЧС извлекли его из воды, после чег...",failed
8,8,1,1.0,"Мы уже говорили, что подрядчики не должны этог...",Результат для нас в приоритете.,"Строители не будут оплачивать работы, которые ...",failed
...,...,...,...,...,...,...,...
240,240,1,1.0,"""Адвокаты обвиняемых попросили оправдать их по...",Защитник хочет снять с обвиняемой обвинение в ...,Заместители обвиняемых в убийстве Сергея Шкапц...,failed
241,241,1,1.0,"Темы, которые сегодня активно обсуждают делега...",Делегаты из Пензенской области успевают поучас...,На проходящем в Пензенской области съезде «Еди...,failed
244,244,1,1.0,Как пояснили в пресс-службе городской админист...,«Сложение потенциалов» - это создание замкнуты...,"По мнению экспертов, в качестве конкурентного ...",failed
247,247,1,1.0,"""Около 200 человек эвакуированы из корпуса фак...",Трансформаторная подстанция загорелась.,В связи с пожаром в подвальном помещении факул...,failed


Again only a handful of samples are valid.

Hypothesis is **rejected**

#### So what?

In reality, in most cases models are attacked unexpectedly: they predict entailment in a *not_entailment* pair after summarisation.

In [ ]:
round(
    summarization_df_t5[(summarization_df_t5["original label"] == 0) & (summarization_df_t5["attacked label"] == 1)].shape[0]
      /summarization_df_t5[summarization_df_t5["attack"] == "succeeded"].shape[0],
      2)

0.91

In [ ]:
round(
    summarization_df_roberta[(summarization_df_roberta["original label"] == 0) & (summarization_df_roberta["attacked label"] == 1)].shape[0]
    /summarization_df_roberta[summarization_df_roberta["attack"] == "succeeded"].shape[0],
    2)

0.82

To test this hypothesis, let's view the probabilities of the model. Maybe if there is no clear opposition in premise and hypothesis, the model gets confused and is not sure in its predictions.

First, create classes for the model and attack.

In [ ]:
import torch

softmax = torch.nn.Softmax(dim=1)

class RobertaModel_custom(RobertaModel):
    def predict(self, inputs):
        with torch.no_grad():
            input_ids = inputs["input_ids"].reshape((1, -1)).to(self.device)
            attention_mask = inputs["attention_mask"].reshape((1, -1)).to(self.device)

            prediction = self.model(input_ids=input_ids, attention_mask=attention_mask).logits
            prediction_label = torch.argmax(prediction, dim=1)
            return int(prediction_label[0]), softmax(prediction)

class Summarization_custom_2(Summarization):
    def attack(self, model, dataset):
        for i, row in dataset.iterrows():
            premise = row["premise"]
            hypothesis = row["hypothesis"]
            label = row["label"]
            if len(premise) < 200:
                continue
            if label == 0:
                prediction, probabilities_orig = model.predict(model.prepare_data(premise, hypothesis))
                if prediction == 0:
                    transformed = self.summarizer(premise, max_length=50)[0]["summary_text"]
                    prediction, probabilities_trans = model.predict(model.prepare_data(premise, transformed))
                    if prediction == 1:
                        print(f"""
    probabilities before: {probabilities_orig}
    probabilities after: {probabilities_trans}
                        """)

In [ ]:
Roberta_costum = RobertaModel_custom(roberta_checkpoints)
summarization_2 = Summarization_custom_2()

View probabilities...

In [ ]:
summarization_2.attack(Roberta_costum, data)


    probabilities before: tensor([[9.9993e-01, 7.4030e-05]], device='cuda:0')
    probabilities after: tensor([[1.9129e-04, 9.9981e-01]], device='cuda:0')
                        

    probabilities before: tensor([[9.9987e-01, 1.3381e-04]], device='cuda:0')
    probabilities after: tensor([[2.2151e-04, 9.9978e-01]], device='cuda:0')
                        

    probabilities before: tensor([[9.9987e-01, 1.3415e-04]], device='cuda:0')
    probabilities after: tensor([[6.7432e-04, 9.9933e-01]], device='cuda:0')
                        

    probabilities before: tensor([[9.9989e-01, 1.0648e-04]], device='cuda:0')
    probabilities after: tensor([[6.2864e-04, 9.9937e-01]], device='cuda:0')
                        

    probabilities before: tensor([[9.9992e-01, 7.5879e-05]], device='cuda:0')
    probabilities after: tensor([[1.8460e-04, 9.9982e-01]], device='cuda:0')
                        

    probabilities before: tensor([[9.9991e-01, 9.3700e-05]], device='cuda:0')
    probabilitie

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:998: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,



    probabilities before: tensor([[9.9964e-01, 3.5564e-04]], device='cuda:0')
    probabilities after: tensor([[1.8555e-04, 9.9981e-01]], device='cuda:0')
                        

    probabilities before: tensor([[9.9993e-01, 7.3876e-05]], device='cuda:0')
    probabilities after: tensor([[4.4515e-04, 9.9955e-01]], device='cuda:0')
                        

    probabilities before: tensor([[0.9939, 0.0061]], device='cuda:0')
    probabilities after: tensor([[0.0360, 0.9640]], device='cuda:0')
                        

    probabilities before: tensor([[9.9993e-01, 7.0832e-05]], device='cuda:0')
    probabilities after: tensor([[2.7441e-04, 9.9973e-01]], device='cuda:0')
                        

    probabilities before: tensor([[9.9984e-01, 1.5845e-04]], device='cuda:0')
    probabilities after: tensor([[2.5919e-04, 9.9974e-01]], device='cuda:0')
                        


Your max_length is set to 50, but you input_length is only 39. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)



    probabilities before: tensor([[9.9991e-01, 9.0342e-05]], device='cuda:0')
    probabilities after: tensor([[3.7956e-04, 9.9962e-01]], device='cuda:0')
                        

    probabilities before: tensor([[9.9992e-01, 8.3653e-05]], device='cuda:0')
    probabilities after: tensor([[6.7999e-04, 9.9932e-01]], device='cuda:0')
                        

    probabilities before: tensor([[9.9993e-01, 7.3472e-05]], device='cuda:0')
    probabilities after: tensor([[3.2901e-04, 9.9967e-01]], device='cuda:0')
                        

    probabilities before: tensor([[0.9987, 0.0013]], device='cuda:0')
    probabilities after: tensor([[2.1749e-04, 9.9978e-01]], device='cuda:0')
                        

    probabilities before: tensor([[9.9987e-01, 1.2912e-04]], device='cuda:0')
    probabilities after: tensor([[2.2591e-04, 9.9977e-01]], device='cuda:0')
                        

    probabilities before: tensor([[9.9986e-01, 1.3558e-04]], device='cuda:0')
    probabilities after:

...


    probabilities before: tensor([[9.9993e-01, 7.3895e-05]], device='cuda:0')
    probabilities after: tensor([[2.6077e-04, 9.9974e-01]], device='cuda:0')
                        

    probabilities before: tensor([[9.9993e-01, 6.6736e-05]], device='cuda:0')
    probabilities after: tensor([[2.1185e-04, 9.9979e-01]], device='cuda:0')
                        

    probabilities before: tensor([[0.8851, 0.1149]], device='cuda:0')
    probabilities after: tensor([[4.4515e-04, 9.9955e-01]], device='cuda:0')
                        

    probabilities before: tensor([[0.9974, 0.0026]], device='cuda:0')
    probabilities after: tensor([[0.0012, 0.9988]], device='cuda:0')
                        

    probabilities before: tensor([[0.6300, 0.3700]], device='cuda:0')
    probabilities after: tensor([[3.3542e-04, 9.9966e-01]], device='cuda:0')
                        

    probabilities before: tensor([[9.9987e-01, 1.2867e-04]], device='cuda:0')
    probabilities after: tensor([[5.7467e-04, 9.

Your max_length is set to 50, but you input_length is only 44. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=22)



    probabilities before: tensor([[9.9990e-01, 1.0429e-04]], device='cuda:0')
    probabilities after: tensor([[1.7941e-04, 9.9982e-01]], device='cuda:0')
                        

    probabilities before: tensor([[9.9993e-01, 6.7767e-05]], device='cuda:0')
    probabilities after: tensor([[1.9045e-04, 9.9981e-01]], device='cuda:0')
                        

    probabilities before: tensor([[0.9069, 0.0931]], device='cuda:0')
    probabilities after: tensor([[0.0040, 0.9960]], device='cuda:0')
                        


Your max_length is set to 50, but you input_length is only 46. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)



    probabilities before: tensor([[9.9993e-01, 6.9057e-05]], device='cuda:0')
    probabilities after: tensor([[3.6939e-04, 9.9963e-01]], device='cuda:0')
                        

    probabilities before: tensor([[9.9951e-01, 4.8727e-04]], device='cuda:0')
    probabilities after: tensor([[1.8901e-04, 9.9981e-01]], device='cuda:0')
                        

    probabilities before: tensor([[0.8404, 0.1596]], device='cuda:0')
    probabilities after: tensor([[3.0786e-04, 9.9969e-01]], device='cuda:0')
                        

    probabilities before: tensor([[9.9993e-01, 6.9135e-05]], device='cuda:0')
    probabilities after: tensor([[0.0039, 0.9961]], device='cuda:0')
                        


... the probabilities are very strong! Before the transformation they are about 0.9 and 0.1, and after it they change in the opposite direction.

### Word Order shuffling attack

In this attack, apart from accuracy and success rate, confusion matrix is analysed, with the help of which recall and FPR are measured

In [ ]:
word_order_attack = WordOrder()

Attack T5

In [ ]:
results_word_order_t5 = word_order_attack.attack(ruT5, data)
word_order_df_t5 = pd.DataFrame(results_word_order_t5)


                  [Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:
                  entailment --> not_entailment
                  original premise: """Решение носит символический характер, так как взыскать компенсацию практически невозможно"", - отмечается в сообщении."
                  original hypothesis: Взыскать компенсацию не получится.
                  
                  transformed: не компенсацию Взыскать получится .
                  
                  

                  [Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:
                  entailment --> entailment
                  original premise: "В суде Центрального района Сочи завершилось рассмотрение уголовного дела в отношении 81-летнего местного жителя, который застрелил обидчика своей дочери. ""Степан Вагин приговорен к трем годам лишения свободы условно с испытательным сроком два года"", - сообщил агентству ""Интерфакс-Юг"" в пятницу адвокат подсудимого Илья Иванов. По его словам, суд сделал снисхожден

Here is a confusion matrix for T5 predictions on attacked dataset. And the corresponding metrics of Recall and FPR

In [ ]:
TP = len(word_order_df_t5[(word_order_df_t5["original label"] == 1) & (word_order_df_t5["attacked label"] == 1)])
FP = len(word_order_df_t5[(word_order_df_t5["original label"] == 0) & (word_order_df_t5["attacked label"] == 1)])
FN = len(word_order_df_t5[(word_order_df_t5["original label"] == 1) & (word_order_df_t5["attacked label"] == 0)])
TN = len(word_order_df_t5[(word_order_df_t5["original label"] == 0) & (word_order_df_t5["attacked label"] == 0)])

matrix_t5 = pd.DataFrame(
    {"True entailment": [TP, FN], "True not_entailment": [FP, TN]}, 
    index=["Predicted entailment", "Predicted not_entailment"]
    )
matrix_t5

,True entailment,True not_entailment
Predicted entailment,124,32
Predicted not_entailment,9,89


In [ ]:
print(f"Recall = {round(TP/(TP + FN), 2)}")
print(f"FPR = {round(FP/(FP + TN), 2)}")

Recall = 0.93
FPR = 0.26


The same for Roberta

In [ ]:
results_word_order_roberta = word_order_attack.attack(ruRoberta, data)
word_order_df_roberta = pd.DataFrame(results_word_order_roberta)


                  [Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:
                  entailment --> entailment
                  original premise: """Решение носит символический характер, так как взыскать компенсацию практически невозможно"", - отмечается в сообщении."
                  original hypothesis: Взыскать компенсацию не получится.
                  
                  transformed: . не получится компенсацию Взыскать
                  
                  

                  [Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:
                  entailment --> entailment
                  original premise: Об этом вечером во вторник, 17 января, сообщила пресс-служба Спасательного департамента, отметив, что немецкую противотанковую мину Tellermine 42 обнаружили в на улице Кеэвисе в ходе земляных работ. Спасатели эвакуировали жителей окрестных домов, офисов и складских помещений. Уничтожать мину на месте не стали, поскольку это угрожало повреждению трассы трубопровода.
     

In [ ]:
TP = len(word_order_df_roberta[(word_order_df_roberta["original label"] == 1) & (word_order_df_roberta["attacked label"] == 1)])
FP = len(word_order_df_roberta[(word_order_df_roberta["original label"] == 0) & (word_order_df_roberta["attacked label"] == 1)])
FN = len(word_order_df_roberta[(word_order_df_roberta["original label"] == 1) & (word_order_df_roberta["attacked label"] == 0)])
TN = len(word_order_df_roberta[(word_order_df_roberta["original label"] == 0) & (word_order_df_roberta["attacked label"] == 0)])

matrix_roberta = pd.DataFrame(
    {"True entailment": [TP, FN], "True not_entailment": [FP, TN]}, 
    index=["Predicted entailment", "Predicted not_entailment"]
    )
matrix_roberta

,True entailment,True not_entailment
Predicted entailment,87,22
Predicted not_entailment,47,101


In [ ]:
print(f"Recall = {round(TP/(TP + FN), 2)}")
print(f"FPR = {round(FP/(FP + TN), 2)}")

Recall = 0.65
FPR = 0.18


Here we did not output examples of successful attacks out of concern of space economy. One can view them in attack outputs or perform themselves.

### Predicate Arguments' shuffling attack

The same manipulations are produced to predicate arguments' shuffling attack.

In [ ]:
change_arguments_attack = ChangeArguments()

T5

In [ ]:
results_change_arguments_t5 = change_arguments_attack.attack(ruT5, data)
change_arguments_df_t5 = pd.DataFrame(results_change_arguments_t5)


                  [Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:
                  entailment --> Skipped

                  

                  [Succeeded / Failed / Skipped / Total] 1 / 0 / 1 / 2:
                  entailment --> entailment
                  original premise: "В суде Центрального района Сочи завершилось рассмотрение уголовного дела в отношении 81-летнего местного жителя, который застрелил обидчика своей дочери. ""Степан Вагин приговорен к трем годам лишения свободы условно с испытательным сроком два года"", - сообщил агентству ""Интерфакс-Юг"" в пятницу адвокат подсудимого Илья Иванов. По его словам, суд сделал снисхождение пенсионеру, поскольку он является тружеником тыла и инвалидом второй группы, и назначил наказание ""ниже низшего предела""."
                  original hypothesis: 81-летний пенсионер застрелил обидчика дочери.
                  
                  transformed: 81-летний обидчик застрелил пенсионера дочери .
                  
             

In [ ]:
TP = len(change_arguments_df_t5[(change_arguments_df_t5["original label"] == 1) & (change_arguments_df_t5["attacked label"] == 1)])
FP = len(change_arguments_df_t5[(change_arguments_df_t5["original label"] == 0) & (change_arguments_df_t5["attacked label"] == 1)])
FN = len(change_arguments_df_t5[(change_arguments_df_t5["original label"] == 1) & (change_arguments_df_t5["attacked label"] == 0)])
TN = len(change_arguments_df_t5[(change_arguments_df_t5["original label"] == 0) & (change_arguments_df_t5["attacked label"] == 0)])

matrix_t5 = pd.DataFrame(
    {"True entailment": [TP, FN], "True not_entailment": [FP, TN]}, 
    index=["Predicted entailment", "Predicted not_entailment"]
    )
matrix_t5

,True entailment,True not_entailment
Predicted entailment,65,8
Predicted not_entailment,6,69


In [ ]:
print(f"Recall = {round(TP/(TP + FN), 2)}")
print(f"FPR = {round(FP/(FP + TN), 2)}")

Recall = 0.92
FPR = 0.1


Roberta

In [ ]:
results_change_arguments_roberta = change_arguments_attack.attack(ruRoberta, data)
change_arguments_df_roberta = pd.DataFrame(results_change_arguments_roberta)


                  [Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:
                  entailment --> Skipped

                  

                  [Succeeded / Failed / Skipped / Total] 0 / 1 / 1 / 2:
                  entailment --> not_entailment
                  original premise: Об этом вечером во вторник, 17 января, сообщила пресс-служба Спасательного департамента, отметив, что немецкую противотанковую мину Tellermine 42 обнаружили в на улице Кеэвисе в ходе земляных работ. Спасатели эвакуировали жителей окрестных домов, офисов и складских помещений. Уничтожать мину на месте не стали, поскольку это угрожало повреждению трассы трубопровода.
                  original hypothesis: На улице Кеэвисе жителей эвакуировали из-за мины.
                  
                  transformed: На жителях Кеэвисе мины эвакуировали из-за улицы .
                  
                  

                  [Succeeded / Failed / Skipped / Total] 0 / 2 / 1 / 3:
                  entailment --> not_ent

In [ ]:
TP = len(change_arguments_df_roberta[(change_arguments_df_roberta["original label"] == 1) & (change_arguments_df_roberta["attacked label"] == 1)])
FP = len(change_arguments_df_roberta[(change_arguments_df_roberta["original label"] == 0) & (change_arguments_df_roberta["attacked label"] == 1)])
FN = len(change_arguments_df_roberta[(change_arguments_df_roberta["original label"] == 1) & (change_arguments_df_roberta["attacked label"] == 0)])
TN = len(change_arguments_df_roberta[(change_arguments_df_roberta["original label"] == 0) & (change_arguments_df_roberta["attacked label"] == 0)])

matrix_roberta = pd.DataFrame(
    {"True entailment": [TP, FN], "True not_entailment": [FP, TN]}, 
    index=["Predicted entailment", "Predicted not_entailment"]
    )
matrix_roberta

,True entailment,True not_entailment
Predicted entailment,34,9
Predicted not_entailment,33,72


In [ ]:
print(f"Recall = {round(TP/(TP + FN), 2)}")
print(f"FPR = {round(FP/(FP + TN), 2)}")

Recall = 0.51
FPR = 0.11


### Yoda Style attack

In this and the following attacks only accuracy and success rate are measured

In [ ]:
yoda_style_attack = YodaStyle()

In [ ]:
results_yoda_style_t5 = yoda_style_attack.attack(ruT5, data)
yoda_style_df_t5 = pd.DataFrame(results_yoda_style_t5)


                  [Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:
                  entailment --> Skipped

                  

                  [Succeeded / Failed / Skipped / Total] 0 / 1 / 1 / 2:
                  entailment --> entailment
                  original premise: "В суде Центрального района Сочи завершилось рассмотрение уголовного дела в отношении 81-летнего местного жителя, который застрелил обидчика своей дочери. ""Степан Вагин приговорен к трем годам лишения свободы условно с испытательным сроком два года"", - сообщил агентству ""Интерфакс-Юг"" в пятницу адвокат подсудимого Илья Иванов. По его словам, суд сделал снисхождение пенсионеру, поскольку он является тружеником тыла и инвалидом второй группы, и назначил наказание ""ниже низшего предела""."
                  original hypothesis: 81-летний пенсионер застрелил обидчика дочери.
                  
                  transformed: Обидчика дочери 81-летний пенсионер застрелил .
                  
             

In [ ]:
results_yoda_style_roberta = yoda_style_attack.attack(ruRoberta, data)
yoda_style_df_roberta = pd.DataFrame(results_yoda_style_roberta)


                  [Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:
                  entailment --> Skipped

                  

                  [Succeeded / Failed / Skipped / Total] 0 / 0 / 2 / 2:
                  entailment --> Skipped

                  

                  [Succeeded / Failed / Skipped / Total] 0 / 1 / 2 / 3:
                  entailment --> entailment
                  original premise: "В суде Центрального района Сочи завершилось рассмотрение уголовного дела в отношении 81-летнего местного жителя, который застрелил обидчика своей дочери. ""Степан Вагин приговорен к трем годам лишения свободы условно с испытательным сроком два года"", - сообщил агентству ""Интерфакс-Юг"" в пятницу адвокат подсудимого Илья Иванов. По его словам, суд сделал снисхождение пенсионеру, поскольку он является тружеником тыла и инвалидом второй группы, и назначил наказание ""ниже низшего предела""."
                  original hypothesis: 81-летний пенсионер застрелил обидчика доче

### Add proverb attack

ruT5

In [ ]:
attack = AddProverb()
results_add_proverb = attack.attack(ruT5, data)
add_proverb_df = pd.DataFrame(results_add_proverb)


                  [Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:
                  entailment --> entailment
                  original premise: """Решение носит символический характер, так как взыскать компенсацию практически невозможно"", - отмечается в сообщении."
                  original hypothesis: Взыскать компенсацию не получится.
                  
                  transformed: """Решение носит символический характер, так как взыскать компенсацию практически невозможно"", - отмечается в сообщении." Без труда не выловишь и рыбку из пруда.
                  
                  

                  [Succeeded / Failed / Skipped / Total] 0 / 2 / 0 / 2:
                  entailment --> entailment
                  original premise: "В суде Центрального района Сочи завершилось рассмотрение уголовного дела в отношении 81-летнего местного жителя, который застрелил обидчика своей дочери. ""Степан Вагин приговорен к трем годам лишения свободы условно с испытательным сроком два г

ruRoberta

In [ ]:
results_add_proverb_roberta = attack.attack(ruRoberta, data)
add_proverb_df_roberta = pd.DataFrame(results_add_proverb_roberta)


                  [Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:
                  entailment --> entailment
                  original premise: """Решение носит символический характер, так как взыскать компенсацию практически невозможно"", - отмечается в сообщении."
                  original hypothesis: Взыскать компенсацию не получится.
                  
                  transformed: """Решение носит символический характер, так как взыскать компенсацию практически невозможно"", - отмечается в сообщении." Без труда не выловишь и рыбку из пруда.
                  
                  

                  [Succeeded / Failed / Skipped / Total] 0 / 2 / 0 / 2:
                  entailment --> entailment
                  original premise: Об этом вечером во вторник, 17 января, сообщила пресс-служба Спасательного департамента, отметив, что немецкую противотанковую мину Tellermine 42 обнаружили в на улице Кеэвисе в ходе земляных работ. Спасатели эвакуировали жителей окрестных домов,